In [1]:
!pip install jsonlines
!pip install tensorflow

import jsonlines
import json
import nltk
nltk.download("stopwords")
import re
import numpy as np
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn import metrics
import collections
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM,Flatten


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir("/content/drive/My Drive") 

In [4]:
raw_train_data = []
with open("project-data/train.data.jsonl", "r+") as f:
    for each_item in jsonlines.Reader(f):
        raw_train_data.append(each_item)

In [5]:
raw_dev_data = []
with open("project-data/dev.data.jsonl", "r+") as f:
    for each_item in jsonlines.Reader(f):
        raw_dev_data.append(each_item)

In [6]:
raw_test_data = []
with open("project-data/test.data.jsonl", "r+") as f:
    for each_item in jsonlines.Reader(f):
        raw_test_data.append(each_item)

In [7]:
with open("project-data/train.label.json", "r+") as f:
    raw_train_label = json.load(f)

In [8]:
with open("project-data/dev.label.json", "r+") as f:
    raw_dev_label = json.load(f)

In [9]:
with open("project-data/test-output.json", "r+") as f:
    sample_test_label = json.load(f)

In [10]:
def text_filter(t):
    t = t.lower()
    t = re.sub(r"(@)\S+", "", t)
    t = re.sub(r'_',' ', t)
    t = re.sub(r'\d','', t)
    t = re.sub(r'\-', ' ', t)
    t = re.sub(r'\.', '', t)
    t = re.sub(r'\\', ' ', t)
    t = re.sub(r'\\x\.+', '', t)
    t = re.sub(r'^_.', '', t)
    t = re.sub(r'^ ', '', t)
    t = re.sub(r' $', '', t)
    t = re.sub(r'\#\.', '', t)
    t = re.sub(r'&', '', t)
    t = re.sub(r'\n', '', t)
    t = re.sub(r',', '', t)
    t = re.sub(r"\n", "", t)
    t = re.sub(r"(https?\://|www)\S+", "", t)
    t = re.sub(r'\?', '', t) 
    t = re.sub(r'it\'s','it is', t)
    t = re.sub(r'i\'m','i am', t)
    t = re.sub(r'she\'s','she is', t)
    t = re.sub(r'he\'s','he is', t)
    t = re.sub(r'that\'s','that is', t)
    t = re.sub(r'they\'re','they are', t)
    t = re.sub(r'ain\'t','am not', t)
    t = re.sub(r'can\'t','can not', t)
    t = re.sub(r'could\'t','could not', t)
    t = re.sub(r'won\'t','will not', t)
    t = re.sub(r'wouldn\'t','would not', t)
    t = re.sub(r'should\'t','should not', t)
    t = re.sub(r'there\'s','there is', t)
    t = re.sub(r'what\'s','what is', t)
    t = t.strip()
    return t

In [12]:
t_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
def raw_data_preprocessing(raw_data):
    data_processed = []
    for each_item in raw_data:
        processed_text_temp = []   
        processed_word = text_filter(each_item)
        processed_word = t_tokenizer.tokenize(processed_word)
        for each_word in processed_word:
            if (len(each_word) != 0) and (re.search(r"[a-z]+", each_word) is not None) and (each_word not in set(stopwords.words('english'))): 
                processed_text_temp.append(each_word)
        data_processed.append(processed_text_temp)
    #print(data_processed[0])    
    return data_processed


In [13]:
raw_train_data_conc = []
for i in raw_train_data:
    each_twitter_text = " "
    for j in i:
        each_twitter_text = each_twitter_text + j['text']
    raw_train_data_conc.append(each_twitter_text)

In [ ]:
each_twitter_text

' Prayers for #Ferguson.@BET @MTV #INMTW http://t.co/s21rhqCKQN@laddygagha @BET hummm“@BET: Prayers for #Ferguson.” prayers for #MikeBrown family@BET http://t.co/nSs0kkAN8t@BET Our thoughts and prayers. #Ferguson“@BET: Prayers for #Ferguson.” http://t.co/52pILjsFPv@BET #Anonymous has released the names@BET AMEN!!!\n#Praying@bet BUY 20K followers for ONLY $50 http://t.co/bmYZrv882s@BET Pray For Us #Ferguson  #JusticeForMikeBrown“@BET: Prayers for #Ferguson.” And his family and friends! Especially the kid with him at the shooting! #prayers #godhelpourcommunities“@BET: Prayers for #Ferguson.” Unhelpful tweets.“@BET: Prayers for #Ferguson.”'

In [14]:
raw_dev_data_conc = []
for i in raw_dev_data:
    each_twitter_text = " "
    for j in i:
        each_twitter_text = each_twitter_text + j['text']
    raw_dev_data_conc.append(each_twitter_text)

In [15]:
raw_test_data_conc = []
for i in raw_test_data:
    each_twitter_text = " "
    for j in i:
        each_twitter_text = each_twitter_text + j['text']
    raw_test_data_conc.append(each_twitter_text)

In [16]:
processed_train_data = raw_data_preprocessing(raw_train_data_conc)
processed_dev_data = raw_data_preprocessing(raw_dev_data_conc)
processed_test_data = raw_data_preprocessing(raw_test_data_conc)

In [17]:
tokenizer = Tokenizer(lower=True,split=',')
tokenizer.fit_on_texts(processed_train_data)
encoded_train_data = tokenizer.texts_to_sequences(processed_train_data)
encoded_dev_data = tokenizer.texts_to_sequences(processed_dev_data)
encoded_test_data = tokenizer.texts_to_sequences(processed_test_data)

In [18]:
# Padding
pad_train_data = pad_sequences(encoded_train_data)
pad_dev_data = pad_sequences(encoded_dev_data, maxlen = len(pad_train_data[0]))
pad_test_data = pad_sequences(encoded_test_data, maxlen = len(pad_train_data[0]))

In [19]:
dev_labels_encoded = []
for i in raw_dev_label.values():
    if i == 'non-rumour':     
        dev_labels_encoded.append(0)     
    else:     
        dev_labels_encoded.append(1)      

In [ ]:
# Oversampling

In [20]:
train_labels_encoded = []
negative_label_indexes = []
positive_label_indexes = []
j=-1
for i in raw_train_label.values():
    j+=1
    if i == 'non-rumour':
        train_labels_encoded.append(0)
        negative_label_indexes.append(j)
    else:
        train_labels_encoded.append(1)
        positive_label_indexes.append(j)
print('Labeled Rumour: Non Rumour', sum(train_labels_encoded)/(len(train_labels_encoded)-sum(train_labels_encoded)))

Labeled Rumour: Non Rumour 0.5176586003924133


In [21]:
oversampled_negative_sample_indexes = np.random.choice(negative_label_indexes,sum(train_labels_encoded),replace=False)
oversampled_training_data=[]
oversampled_training_labels=[]

for each_index in oversampled_negative_sample_indexes:
      oversampled_training_labels.append(train_labels_encoded[each_index])
      oversampled_training_data.append(pad_train_data[each_index])

for each_index in positive_label_indexes:
      oversampled_training_labels.append(train_labels_encoded[each_index])
      oversampled_training_data.append(pad_train_data[each_index])

sum(oversampled_training_labels)/len(oversampled_training_labels)

0.5

In [23]:
np_train_data = np.append(pad_train_data, np.array(oversampled_training_data), axis=0)
np_train_labels = np.append(train_labels_encoded, np.array(oversampled_training_labels), axis=0)
dev_labels_encoded = np.array(dev_labels_encoded)

In [24]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(None,), dtype="int32"),
    tf.keras.layers.Embedding(2000000, 128),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [25]:
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [26]:
len(dev_labels_encoded)

580

In [27]:
len(pad_dev_data)

580

In [28]:
model.save('./lstm_Concatenated.h5')

In [29]:
model.fit(np_train_data, np_train_labels, batch_size=128, epochs=20, validation_data=(pad_dev_data, dev_labels_encoded))

Epoch 1/20
61/61 [==============================] - 543s 9s/step - loss: 0.6345 - accuracy: 0.6343 - val_loss: 0.4527 - val_accuracy: 0.8207
Epoch 2/20
61/61 [==============================] - 537s 9s/step - loss: 0.2196 - accuracy: 0.9251 - val_loss: 0.7309 - val_accuracy: 0.8155
Epoch 3/20
61/61 [==============================] - 550s 9s/step - loss: 0.0472 - accuracy: 0.9889 - val_loss: 1.1334 - val_accuracy: 0.7914
Epoch 4/20
61/61 [==============================] - 558s 9s/step - loss: 0.0196 - accuracy: 0.9963 - val_loss: 1.1918 - val_accuracy: 0.8000
Epoch 5/20
61/61 [==============================] - 559s 9s/step - loss: 0.0148 - accuracy: 0.9974 - val_loss: 1.2821 - val_accuracy: 0.8017
Epoch 6/20
61/61 [==============================] - 562s 9s/step - loss: 0.0050 - accuracy: 0.9989 - val_loss: 1.2805 - val_accuracy: 0.7983
Epoch 7/20
61/61 [==============================] - 561s 9s/step - loss: 0.0037 - accuracy: 0.9992 - val_loss: 1.5269 - val_accuracy: 0.8017
Epoch 8/20
61

In [31]:
dev_pre_freq = model.predict(pad_dev_data)

In [32]:
test_pre_freq = model.predict(pad_test_data)

In [35]:
dev_pred = []
for i in dev_pre_freq:
    if i > 0.5:
        dev_pred.append(1)
    else:
        dev_pred.append(0)
dev_pred = np.array(dev_pred)        

In [36]:
test_pred = []
for i in test_pre_freq:
    if i > 0.5:
        test_pred.append(1)
    else:
        test_pred.append(0)
test_pred = np.array(test_pred)    

In [39]:
print(metrics.classification_report(dev_labels_encoded, dev_pred))

              precision    recall  f1-score   support

           0       0.83      0.90      0.87       393
           1       0.75      0.63      0.68       187

    accuracy                           0.81       580
   macro avg       0.79      0.76      0.77       580
weighted avg       0.81      0.81      0.81       580



In [43]:
test_pred_json = dict()
for i in range(len(sample_test_label.keys())):
    if test_pred[i] == 1:
        label_value = 'rumour'
    else:
        label_value = 'non-rumour'
    test_pred_json[list(sample_test_label.keys())[i]] = label_value

In [45]:
with open("test-output-2.json","w") as f:  
    json.dump(dict(test_pred_json),f)
    print("test-output-2.json Saved Successfully!")

test-output-2.json Saved Successfully!
